# Uniswap V3 OHCLV candle price analysis

In this notebook we will show how to download price events from Uniswap V3 to your computer as CSV files and use them to analyse price in each pool.
This will generate an [OHLCV price chart](https://tradingstrategy.ai/glossary/ohlcv).

- The notebook will fetch price data for all pairs on Uniswap v3 Ethereum which traded at a specific period, or a block range.

- Running this notebook will do ~1M API requests on your JSON-RPC provider. You can reduce the scanned block range to decrease the number of API requests.

- Running this notebook will be slow (hours), depending how high quality your JSON-RPC provider is. With a local JSON-RPC node it completes in few minutes.

## Prerequisites

* For more background information, see [this blog post about the topic](https://tradingstrategy.ai/blog/uniswap-data-research-with-jupyter-and-python)

* [See here a tutorial how to set up your local development environment to run this notebook](https://tradingstrategy.ai/docs/programming/setting-up-development-environment/index.html)

* For easier access to data, see also [Trading Strategy historical datasets](https://tradingstrategy.ai/trading-view/backtesting)
  where you can batch download historical data in one file

* You need to understand [Jupyter Notebook and Pandas basics](https://jakevdp.github.io/PythonDataScienceHandbook/)

* You need to understand [Ethereum](https://github.com/ethereumbook/ethereumbook) and [Web3.py basics](https://web3py.readthedocs.io/)

* You know how to use [Plotly charting library for Python](https://plotly.com/)

* You will need to have [Ethereum API node and its JSON-RPC URL](https://ethereumnodes.com/) in order to pull out the data from Ethereum blockchain. The notebook will interactively ask you for your API key.

* You will need to install [web3-ethereum-defi](https://github.com/tradingstrategy-ai/web3-ethereum-defi) Python package

* This notebook uses UNIX style paths and may not run on Microsoft Windows unless modified.

### Running

Run the notebook in Visual Studio Code or similar.

To run from the command line using IPython command:

```shell
ipython docs/source/tutorials/uniswap-v3-price-analysis.ipynb
```

## Extracting and transforming the data

This will extract Uniswap v3 events, save them in temporary CSV files,
and then transforms them to OHLCV data.

### Download the raw data from Ethereum blockchain

You can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/). Long term, we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node).

In [1]:
from web3 import Web3, HTTPProvider

# Get your node JSON-RPC URL
# interactively when you run the notebook.
# The actual UI prompt will depend on your environment (Visual Studio Code, Jupyter Notebook, etc.)
json_rpc_url = input("Please enter your Ethereum mainnet JSON-RPC URL here: ")
web3 = Web3(HTTPProvider(json_rpc_url))

# Configure logging for diagnostics if needed
#import logging
#import sys
#logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

As an example, here we download raw events from first few blocks after Uniswap V3 was deployed.
The events will be stored in several CSV files at `/tmp` folder.

Depends on your internet connection and latency to the Ethereum node, the scan might take hours. However it can resume in case there is a crash, as we save the last scanned block in a JSON state file.

[See the source code of fetch_events_to_csv](https://web3-ethereum-defi.readthedocs.io/_modules/eth_defi/uniswap_v3/events.html#fetch_events_to_csv).

**Note**: Progress bar might be slow to update at the start.

In [2]:
from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv
from eth_defi.event_reader.json_state import JSONFileScanState

# Take a snapshot of 250,000 blocks after Uniswap v3 deployment
start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 250_000

# Stores the last block number of event data we store
state = JSONFileScanState("/tmp/uniswap-v3-price-scan.json")

print(f"Data snapshot range set to {start_block:,} - {end_block:,}")

# Load the events and write them into a CSV file.
# Several different CSV files are created,
# each for one event type: swap, pool created, mint, burn
web3 = fetch_events_to_csv(
    json_rpc_url,
    state,
    start_block=start_block,
    end_block=end_block,
    output_folder="/tmp",
    # Configure depending on what's eth_getLogs
    # limit of your JSON-RPC provider
    max_blocks_once=222,
    # Do reading and decoding in parallel threads
    max_threads=8,
)



Restored previous scan state, data until block 12,375,721, we are skipping 6,100 blocks out of 250,000 total
Scanning block range 12,375,721 - 12,619,621


  0%|          | 0/243900 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
## JSON-RPC API usage

Show how many API calls we made to our JSON-RPC provider.


In [ ]:
import pandas as pd

api_call_counts = web3.get_api_call_counts()

data = [(k, v) for k, v in api_call_counts.items()]
df = pd.DataFrame(data, columns=["endpoint", "calls",])
df

### Analysing Uniswap v3 price formation

In Uniswap V3, you can get the current price of any pool from any given moment using swap events.

In [ ]:
swap_df = pd.read_csv("/tmp/uniswap-v3-swap.csv")

print(f"We have total {len(swap_df):,} swaps in the dataset")

Choose a pool to analyse, for example: USDC/ETH 0.3%.
You can find pool addresses [on Uniswap v3 info site](https://info.uniswap.org/#/).

In [ ]:
from eth_defi.uniswap_v3.pool import fetch_pool_details

pool_address = "0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8"
pool_details = fetch_pool_details(web3, pool_address)

print(pool_details)
print("token0 is", pool_details.token0, "with", pool_details.token0.decimals, "decimals")
print("token1 is", pool_details.token1, "with", pool_details.token1.decimals, "decimals")

Extract the swaps of this pool from the Uniswap v3 full dataset of all pools.

In [ ]:
df = swap_df.loc[swap_df.pool_contract_address == pool_address.lower()]
df

In Uniswap V3, we can get pool's price from pool's tick.

In [ ]:
def convert_price(row):
    # USDC/WETH pool has reverse token order, so let's flip it WETH/USDC
    tick = row["tick"]
    return pool_details.convert_price_to_human(tick, reverse_token_order=True)

def convert_value(row):
    # USDC is token0 and amount0
    price = float(row["price"])
    return abs(float(row["amount0"])) / (10**pool_details.token0.decimals)

df = df.copy(deep=True)  # https://stackoverflow.com/a/60885847/315168
df["price"] = df.apply(convert_price, axis=1)
df["value"] = df.apply(convert_value, axis=1)
df[["block_number", "timestamp", "tick", "price", "value"]]

Then we can convert linear price data to [OHLC candles](https://tradingstrategy.ai/docs/glossary.html#term-OHLCV).

In [ ]:
from eth_defi.research.candle import convert_to_ohlcv_candles

candles = convert_to_ohlcv_candles(df, time_bucket=pd.Timedelta("4h"))

# Show only 100 first candles
candles = candles.head(100)

candles

### Drawing OHLCV chart

Now we can plot the [OHLC chart using Plotly](https://plotly.com/python/ohlc-charts/).

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

candlesticks = go.Candlestick(
    x=candles.index,
    open=candles['open'],
    high=candles['high'],
    low=candles['low'],
    close=candles['close'],
    showlegend=False
)

volume_bars = go.Bar(
    x=candles.index,
    y=candles['volume'],
    showlegend=False,
    marker={
        "color": "rgba(128,128,128,0.5)",
    }
)

fig = go.Figure(candlesticks)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(candlesticks, secondary_y=True)
fig.add_trace(volume_bars, secondary_y=False)
fig.update_layout(title="ETH/USDC pool after Uniswap v3 deployment", height=800)
fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
fig.show()